In [ ]:
!pip install insightface onnxruntime tensorrt tensorrt-bindings tenseal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 980.8/980.8 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 118.0 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [14]:
import insightface
import numpy as np
from PIL import Image

# Load model
model = insightface.app.FaceAnalysis(name='buffalo_l')
model.prepare(ctx_id=0)  # 0 for GPU, -1 for CPU

def get_embedding(image_path):
    img = Image.open(image_path).convert("RGB")
    img = np.array(img)
    faces = model.get(img)
    if len(faces) == 0:
        raise ValueError("No face detected.")
    return faces[0].embedding  # 512-d float32 vector


/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [15]:
import tenseal as ts

def get_context():
    # 16384 is secure, 8192 is faster but weaker
    context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
    context.generate_galois_keys()
    context.global_scale = 2**40
    return context

def encrypt_embedding(context, embedding):
    return ts.ckks_vector(context, embedding.tolist())

In [16]:
# enrollment
context = get_context()
embedding = get_embedding("teste.jpg")

# Storing the norm like this is necessary to avoid using the decrypt_vector function

embedding_norm = np.linalg.norm(embedding)
enc_embedding = encrypt_embedding(context, embedding)
stored_template = enc_embedding.serialize()  # Save this in a file or database

In [18]:
# We need the norm (L2 length) of the stored embedding
# We can decrypt it just to get the norm, but it breaks the purpose of using FHE
# For now, I'm storing the norm in a variable and can maybe encrypt it aswell

def decrypt_vector(context, serialized_vec):
    vec = ts.ckks_vector_from(context, serialized_vec)
    return np.array(vec.decrypt())

def cosine_similarity(vec1, vec2):
    # plain cosine similarity
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Load and encrypt probe image
probe_embedding = get_embedding("heitor_verify.jpg")
enc_probe = encrypt_embedding(context, probe_embedding)

# Deserialize stored template and compute similarity
enc_template = ts.ckks_vector_from(context, stored_template)

# Element-wise product
product = enc_template * enc_probe
similarity = sum(product.decrypt())  # approx dot product

# Normalize (approximate)
score = similarity / (np.linalg.norm(probe_embedding) * np.linalg.norm(embedding_norm))

# Decision
print("Match!" if score > 0.8 else "No match")
print(f"Score: {score}")

Match!
Score: 0.9687895774841309
